In [ ]:
import sys  
sys.path.insert(0, './')
import torch
import numpy as np
from neural_interaction_detection import get_interactions
from multilayer_perceptron import MLP, train, get_weights
from utils import preprocess_data, get_pairwise_auc, get_anyorder_R_precision, set_seed, print_rankings

In [ ]:
use_main_effect_nets = False # toggle this to use "main effect" nets
num_samples = 10000
num_features = 100

## Train a multilayer perceptron (MLP)

In [ ]:
def get_precision(interactions, ground_truth):

    R = len(ground_truth)
    recovered_gt = []
    counter = 0

    for inter, strength in interactions:
        if counter == R:
            break

        inter_set = set(inter)
        if any(inter_set <= gt for gt in ground_truth):
            counter += 1
    R_precision = counter / R

    return R_precision

In [ ]:
Rs=[]
x_test=np.genfromtxt("../x_testF3b.csv")
y_test=np.genfromtxt("../y_testF3b.csv")
gt=[{1,2},{3,4},{5,6},{5,8},{6,8},{8,9},{8,10},{9,10}]

loss=[]
print(x_test.shape[1])
for i in range(10):
    x_train=np.genfromtxt("../x_trainF3"+str(i)+"b.csv")
    y_train=np.genfromtxt("../y_trainF3"+str(i)+"b.csv")
    x_val=np.genfromtxt("../x_valF3"+str(i)+"b.csv")
    y_val=np.genfromtxt("../y_valF3"+str(i)+"b.csv")
    data_loaders = preprocess_data(x_train, y_train,x_val,y_val,x_test,y_test, std_scale=False, get_torch_loaders=True)
    device = torch.device("cpu")
    model = MLP(x_train.shape[1], [140, 100, 60, 20], use_main_effect_nets=use_main_effect_nets).to(device)
    model, mlp_loss = train(model, data_loaders, device=device,nepochs=150, learning_rate=1e-2, l1_const = 1e-5, verbose=True,early_stopping=False)
    model_weights = get_weights(model)
    print(mlp_loss)
    pairwise_interactions = get_interactions(model_weights, pairwise=True, one_indexed=True)
    r_prec = get_precision(pairwise_interactions[:len(gt)], gt)
    print("Precision", r_prec,pairwise_interactions[:len(gt)])
    Rs.append(r_prec)

In [ ]:
mean=np.mean(Rs)
sd=np.std(Rs)
print(mean,sd)